In [ ]:
from utils import utils_models, utils_gn, utils_dgrd, utils_ivc, utils_noah
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import importlib
import numpy as np
from config.definitions import ROOT_DIR
import seaborn as sns
importlib.reload(utils_models)
importlib.reload(utils_gn)
importlib.reload(utils_dgrd)
importlib.reload(utils_ivc)
importlib.reload(utils_noah)

In [ ]:
# read training data
train_raw_data = utils_gn.read_data(
    path=f"{ROOT_DIR}/data",
    fname="train_1238.pkl"
)
train_raw_data.keys()

In [ ]:
len(train_raw_data['b2c12']['cycle_dict'])

In [ ]:
# Check the knee/elbow detection algorithm on sample cells

# For knee
sample_cells = ['b1c7', 'b2c42', 'b3c26']
for sample_cell in sample_cells:
    Qd = train_raw_data[sample_cell]['summary']['QDischarge']
    utils_models.knee_elbow_detection(
        x_data=np.arange(len(Qd)) + 1,
        y_data=Qd,
        type='knee',
        p0=None,
        p0_db=None,
        plot=True,
        ylabel='Capacity (Ah)',
        ylim=[.88, 1.1],
        title=sample_cell,
        point_name1='Knee point',
        point_name2='Knee onset'
    )

In [ ]:
# For elbow
for sample_cell in sample_cells:
    IR = train_raw_data[sample_cell]['summary']['IR']
    utils_models.knee_elbow_detection(
        x_data=np.arange(len(IR)) + 1,
        y_data=IR,
        type='elbow',
        p0=None,
        p0_db=None,
        plot=True,
        ylabel='Internal resistance ($\Omega$)',
        ylim=[0.013, 0.023],
        title=sample_cell,
        point_name1='Elbow point',
        point_name2='Elbow onset'
    )

In [ ]:
# Generate training data frame for n=50 and n=100
knee_elbow_df = utils_ivc.ccv_features(
    data_dict=train_raw_data,
    n=50
).join(utils_dgrd.create_knee_elbow_data(train_raw_data))

knee_elbow_df2 = utils_ivc.ccv_features(
    data_dict=train_raw_data,
    n=100
).join(utils_dgrd.create_knee_elbow_data(train_raw_data))

In [ ]:
# Checking correlation: heat map for n=50 and n=100
targets = [
    'k-o', 'k-p', 'Qatk-o', 'Qatk-p',
    'e-o', 'e-p', 'IRate-o', 'IRate-p',
    'IRatEOL', 'EOL'
]

for df, fname in zip((knee_elbow_df, knee_elbow_df2), ('correlation-heatmap-n-50', 'correlation-heatmap-n-100')):

    corr_matrix = df.corr()

    features = [el for el in df.columns if el not in targets]
    features.remove('RUL')

    corr_for_heatmap = corr_matrix.loc[targets, features]
    fig, ax = plt.subplots(figsize=(4, 18))
    cax = inset_axes(
        ax,
        width="100%",
        height="2%",
        loc='lower left',
        bbox_to_anchor=(0, -0.09, 1, 1),
        bbox_transform=ax.transAxes,
        borderpad=0,
    )
    ax.set_xticklabels(targets, fontsize=14)
    ax.set_yticklabels(features, fontsize=14)
    sns.heatmap(corr_for_heatmap.T, vmin=-1, vmax=1, xticklabels=targets, yticklabels=features, linewidth=1, cmap='seismic', alpha=0.7, linecolor='black', ax=ax,
                cbar_ax=cax, cbar_kws={'orientation': 'horizontal', 'label': 'Correlation'})
    ax.figure.axes[-1].set_xlabel(r'Correlation ($\rho$)', size=14)

    plt.savefig(fname=f"{ROOT_DIR}/plots/{fname}", bbox_inches='tight')

In [ ]:
# Plot cc discharge voltage for selected cells
data_all = utils_gn.read_data(
    path=f"{ROOT_DIR}/data",
    fname="data_all.pkl"
)
# data_all = {k: data_all[k] for k in train_raw_data.keys()}  # to use data from training set only
ccv = utils_ivc.ccv_features(data_all, return_ccv=True)

In [ ]:
# Save the generated ccv as pkl
utils_gn.dump_data(
    data=ccv,
    path=f"{ROOT_DIR}/data",
    fname= "ccv_data.pkl"
)

In [ ]:
# ccv = utils_gn.read_data('ccv_data.pkl')
selected_cells = ['b1c30', 'b2c30', 'b3c27', 'b8c7']
utils_ivc.plot_CCV_features(ccv, sample_cells=selected_cells, option=2)

In [ ]:
I = train_raw_data['b1c30']['cycle_dict']['2']['I']
V = train_raw_data['b1c30']['cycle_dict']['2']['V']
t = train_raw_data['b1c30']['cycle_dict']['2']['t']
t_discharge = np.logical_and((t >= 31), (t <= 45.5))


ft = [r'Current ($A$)', r'Voltage (V)', r'Temperature ($^{\circ}C$)']
fig, ax = plt.subplots(1, 2, figsize=(11, 3))
for i, f, val in zip([0, 1], ft, [I, V]):

    ax[i].plot(t, val)

    if i == 0:
        ax[i].hlines(y=I[t_discharge][-1], xmin=31, xmax=45.5,
                     linewidth=4, color='red', zorder=100, label='Discharge CC')

    if i == 1:
        ax[i].plot(t[t_discharge], val[t_discharge], linewidth=4,
                   color='red', label='CC discharging voltage')

    ax[i].axvline(x=31, color='red', linestyle='--')
    ax[i].axvline(x=45.5, color='red', linestyle='--')
    ax[i].set_xlabel('Time (minutes)', fontsize=16)
    ax[i].set_ylabel(f, fontsize=16)

    ax[i].legend(loc='lower left')

plt.savefig(fname=f"{ROOT_DIR}/plots/cc-ccdv", bbox_inches='tight')

### Generating figures for reviewers' comments 

In [ ]:
all_batches = utils_gn.read_data(
    fname="data_all.pkl",
    path=f"{ROOT_DIR}/data"
)

In [ ]:
all_batches['b2c1']['charge_policy']

In [ ]:
sample_cells = ['b1c30', 'b1c0', 'b2c3', 'b2c4', 'b3c10', 'b8c4']
# sample_cells = ['b1c30', 'b2c3', 'b3c10', 'b8c4']

fig = plt.figure(figsize=(10, 12), constrained_layout=True)
subfigs = fig.subfigures(6, 1)

for i, subfig in enumerate(subfigs):
    axes = subfig.subplots(1, 2)
    axes[0].plot(all_batches[sample_cells[i]]['cycle_dict']['2']
                 ['t'], all_batches[sample_cells[i]]['cycle_dict']['2']['I'])
    axes[1].plot(all_batches[sample_cells[i]]['cycle_dict']['2']
                 ['t'], all_batches[sample_cells[i]]['cycle_dict']['2']['V'])

    if i == 5:
        axes[0].set_xlabel('Time (minutes)', fontsize=14)
        axes[1].set_xlabel('Time (minutes)', fontsize=14)

    axes[0].set_ylabel(r'Current ($I$)', fontsize=14)
    axes[1].set_ylabel(r'Voltage ($V$)', fontsize=14)

    subfig.suptitle(
        f"Cell: {sample_cells[i]}, cycle: 2, charging policy: {all_batches[sample_cells[i]]['charge_policy']}",
        fontsize=16,
    )